<a href="https://colab.research.google.com/github/changyong93/Natural-language-processing-with-chat-bot/blob/main/%EB%94%A5%EB%9F%AC%EB%8B%9D%EC%9D%84_%EC%9D%B4%EC%9A%A9%ED%95%9C_%EC%9E%90%EC%97%B0%EC%96%B4%EC%B2%98%EB%A6%AC_%EC%9E%85%EB%AC%B8(11_6_RNN%EC%9D%84_%EC%9D%B4%EC%9A%A9%ED%95%9C_%ED%85%8D%EC%8A%A4%ED%8A%B8%EB%B6%84%EB%A5%98_%EB%84%A4%EC%9D%B4%EB%B2%84_%EC%98%81%ED%99%94_%EB%A6%AC%EB%B7%B0_%EA%B0%90%EC%84%B1_%EB%B6%84%EB%A5%98).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 네이버 영화 리뷰 감성 분류하기(Naver Movie Review Sentiment Analysis)

## 네이버 영화 리뷰 데이터에 대한 이해와 전처리
- 다운로드 링크 : https://github.com/e9t/nsmc/

In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import re
import urllib.request

!pip install konlpy
from konlpy.tag import Okt
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

## 데이터 로드

In [ ]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt", filename="ratings_train.txt")
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt", filename="ratings_test.txt")

In [ ]:
train_data = pd.read_table("ratings_train.txt")
test_data = pd.read_table("ratings_test.txt")

In [ ]:
#훈련용 개수 출력
print("훈련용 데이터 개수: ", len(train_data))

In [ ]:
#상위 5개 항목 출력
train_data.head(5)

In [ ]:
#테스트 데이터 개수
print("테스트 데이터 개수: ", test_data.shape[0])

In [ ]:
#테스트 데이터 상위 5개
test_data.head()

## 데이터 정제

In [ ]:
print(train_data.shape)
print("*"*100)
print(train_data.nunique())

In [ ]:
train_data = train_data.drop_duplicates(subset = ["document"]).copy()
train_data.shape

In [ ]:
train_data.label.value_counts().plot.bar()

In [ ]:
train_data.groupby("label").size().reset_index(name = "count")

### 결측치 처리

In [ ]:
#결측치 확인
train_data.isnull().any()

In [ ]:
train_data.isnull().sum()

In [ ]:
train_data[train_data.document.isnull()]

In [ ]:
#결측치 제거
train_data = train_data.dropna(how = "any").copy()
print(train_data.shape)
print(train_data.isnull().any())

### 데이터 전처리 시작

In [ ]:
#알파벳과 공백을 제외하고 제거
train_data["document"] = train_data["document"].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
train_data.head(10)

In [ ]:
train_data.tail(10)

In [ ]:
# 만약 리뷰가 영어, 숫자, 특수문자로 되어있더라면 현재 공백(white space)만 있거나 빈 값을 가진 행이 생성됨
# 상기 내용에 대해 다시 한 번 전처리하여 Null값 처리 후 재확인
print(train_data.isnull().sum())
train_data[train_data.document.str.contains("^ +")].head(5)

In [ ]:
train_data["document"] = train_data["document"].str.replace("^ +", "").copy()
train_data["document"] = train_data["document"].replace("", np.nan)
print(train_data.isnull().sum())

In [ ]:
train_data[train_data.document.isnull()].head(5)

In [ ]:
#Null 샘플은 데이터 분석에 도움이 안되므로 삭제
print("결측치 제거 전 데이터 개수: ", train_data.shape[0])
train_data = train_data.dropna(how = 'any').copy()
print("결측치 제거 후 데이터 개수: ", train_data.shape[0])

In [ ]:
#테스트 데이터도 동일 과정 진행
test_data = test_data.drop_duplicates(subset = ["document"]).copy()
test_data["document"] = test_data["document"].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","").copy()
test_data["document"] = test_data["document"].replace("^ +", "").copy()
test_data["document"] = test_data["document"].replace("", np.nan).copy()
test_data = test_data.dropna(how = "any")

In [ ]:
print(test_data.shape)
print("*"*100)
test_data.isnull().sum()

### 토큰화

In [ ]:
#stopwords 지정
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']

#형태소 분석기 Okt
coupus = '와 이런 것도 영화라고 차라리 뮤직비디오를 만드는 게 나을 뻔'
okt = Okt()
print(okt.morphs(coupus))
print(okt.morphs(coupus,stem = True)) #stem True 시 단어를 보다 정규화 시켜줌

In [ ]:
X_train = []
for sentence in train_data.document:
  temp_x = okt.morphs(sentence, stem = True)
  temp_x = [word for word in temp_x if word not in stopwords]
  X_train.append(temp_x)

In [ ]:
print(X_train[:3])

In [ ]:
X_test = []
for sentence in test_data.document:
  temp_x = okt.morphs(sentence, stem = True)
  temp_x = [word for word in temp_x if word not in stopwords]
  X_test.append(temp_x)

## 정수 인코딩

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)

In [ ]:
print(tokenizer.word_index)

In [ ]:
threshold = 3
total_cnt = len(tokenizer.word_index)
rare_cnt = 0 #등장 빈도수가 threshold보다 적은 단어 개수 카운트hreshold = 3
total_freq = 0 # 훈련 데이터셋의 전체 단어 빈도수 합
rare_freq = 0 #훈련 데이터셋에 threshold보다 등장 빈도수가 적은 단어 빈도수 합

for key,value in tokenizer.word_counts.items():
  total_freq += value #단어별 빈도수를 모두 합침

  #만약 등장 빈도수가 threshold보다 작으면
  if value < threshold:
    rare_cnt +=1
    rare_freq += value

print('단어 집합(vocabulary) 크기: ',total_cnt)
print("등장 빈도수가 %s 이하인 희귀 단어의 수: %s" %(threshold-1, rare_cnt))
print(f"전체 집합에서 희귀단어 비율: {rare_cnt/total_cnt*100:.2f}%")
print(f"전체 집합에서 희귀단어 등장 빈도 비율: {rare_freq/total_freq*100:.2f}%")

In [ ]:
#등장빈도수 2이하인 단어 제거
vocab_size = total_cnt - rare_cnt + 1
print("단어 집합의 크기: ",vocab_size)

In [ ]:
# 단어 집합의 크기를 조정하여 텍스트 시퀀스를 토큰 시퀀스로 변환
#정수인코딩

tokenizer = Tokenizer(num_words = vocab_size)
tokenizer.fit_on_texts(X_train)
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

In [ ]:
print(X_train[:3])

In [ ]:
y_train = np.array(train_data["label"])
y_test = np.array(test_data["label"])

## 빈 샘플 생성
---
- 빈도수가 낮은 단어를 삭제하여, 샘플별로 빈 값이 생성됨
- 빈 단어를 삭제하는 작업 진행

In [ ]:
drop_train = [index for index, sentence in enumerate(X_train) if len(sentence) < 1]

In [ ]:
#빈 샘플 데이터 확인
X_train = np.delete(X_train, drop_train, axis = 0)
y_train = np.delete(y_train, drop_train, axis = 0)

print(X_train.shape[0],y_train.shape[0])

## padding

In [ ]:
print("리뷰의 최대 길이: ", max([len(sentence) for sentence in X_train]))
print("리뷰의 평균 길이: ", np.mean([len(sentence) for sentence in X_train]))

plt.hist([len(sentence) for sentence in X_train], bins = 50)
plt.xlabel("length of samples")
plt.ylabel("number of samples")
plt.show()

In [ ]:
max_len = max([len(sentence) for sentence in X_train])


for lens in range(1,max_len+1):
  cnt = 0
  for index in range(len(X_train)):
    if len(X_train[index]) <= lens:
      cnt += 1
  print(f"전체 샘플 중 길이가 {lens} 이하인 샘플 비율: {cnt / len(X_train) *100 :.3f}")

In [ ]:
#샘플 길이 30으로 제한
max_len = 30
X_train = pad_sequences(sequences = X_train,maxlen = max_len)
X_test = pad_sequences(sequences = X_test, maxlen = max_len)

## LSTM으로 네이버 영화 리뷰 감성 분류

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [ ]:
model = Sequential()
model.add(Embedding(input_dim = vocab_size, output_dim = 100))
model.add(LSTM(units = 128))
model.add(Dense(units = 1, activation = 'sigmoid'))

In [ ]:
es = EarlyStopping(monitor = 'val_loss',patience = 4, verbose = 1, mode = "min")
mc = ModelCheckpoint(filepath = "base_model.h5",monitor = 'val_acc',verbose = 1, mode = "max", save_best_only=True)

In [ ]:
model.compile(optimizer = 'rmsprop', loss = 'binary_crossentropy', metrics = ['acc'])
history = model.fit(x = X_train, y = y_train, batch_size = 60, epochs = 15, validation_split = 0.2, callbacks = [es,mc])

In [ ]:
loaded_model = load_model("base_model.h5")
print(f"\n 테스트 정확도: {loaded_model.evaluate(X_test,y_test)[1]:.4f}")

## 리뷰 예측하는 함수 생성

In [ ]:
def sentiment_predict(new_sentence):
  new_sentence = okt.morphs(new_sentence, stem = True) #토큰화
  new_sentence = [word for word in new_sentence if word not in stopwords]#불용어 제거
  encoded = tokenizer.texts_to_sequences([new_sentence])#정수인코딩
  padding = pad_sequences(encoded, maxlen = max_len)#패딩
  score = float(loaded_model.predict(padding))
  if (score > 0.5):
    print(f"{score*100 :.3f}% 확률로 긍정 리뷰입니다")
  else:
    print(f"{(1-score)*100:.3f}% 확률로 부정 리뷰입니다")



In [ ]:
sentiment_predict('이 영화 개꿀잼 ㅋㅋㅋ')

In [ ]:
sentiment_predict('이 영화 핵노잼 ㅠㅠ')

In [ ]:
sentiment_predict('이딴게 영화냐 ㅉㅉ')

In [ ]:
sentiment_predict('감독 뭐하는 놈이냐?')

In [ ]:
sentiment_predict('와 개쩐다 정말 세계관 최강자들의 영화다')